# C16: Reduce Overfitting With Dropout Regularization

Summary:

    1. from keras.layers import Dropout
    2. model.add(Dense(0.2)) # means 20% of neurons will be excluded.
    3. model.add(Dense(0.2, input_shape=(60,)))  # if dropout layer is input layer

Reference:

    [1] Srivastava, Hinton, et,al, Dropout: A Simple Way to Prevent Neural Networks from Overfitting, 2014

## 1. Preparation

In [3]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

# fix random seed
seed = 7
np.random.seed(seed)

# load data
dataframe = pd.read_csv("./data_set/sonar.data", header=None)
dataset = dataframe.values
X = dataset[:, 0:60].astype(float)
Y = dataset[:, 60]
encoder = LabelEncoder()
encoder.fit(Y)
Y_enc = encoder.transform(Y)

## 2.Define a Model With Dropout Layer

In [4]:
# define a model
def create_nn_with_dropout():
    model = Sequential()
    model.add(Dropout(0.2, input_shape=(60,)))
    model.add(Dense(60, init='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(30, init='normal', activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

## 3. Evaluate Model

In [11]:
model = KerasClassifier(build_fn=create_nn_with_dropout, nb_epoch=150, batch_size=10, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(model, X, Y_enc, cv=kfold)
print("10-fold-val Acc: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

10-fold-val Acc: 83.64% (5.75%)
